In [118]:
# Importing Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException


import pandas as pd

In [119]:
# Setting Chrome Options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")

In [120]:
# Main page link
link = 'https://forteplus-sistemas.gitbook.io/forteplus-sistemas'

In [121]:
# Functions
def setup(link=''):  # Só deve ser executado uma vez.
    '''
    Setup function to initialize the WebDriver and open the specified link.
    '''
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(link)
    wait = WebDriverWait(driver, 15)
    return driver, wait


def verify_end_of_content_page(wait):
    xpath = "//a[contains(@class, 'group') and .//span[text()='Próximo']]"
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    except StaleElementReferenceException:
        wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        return True
    
    return False


def navigate_to_next_page(wait):
    xpath = "//a[contains(@class, 'group') and .//span[text()='Próximo']]"
    try:
        button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        button.click()
    except StaleElementReferenceException:
        # O botão mudou --> buscar de novo
        button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        button.click()
    except:
        print("Não há mais botão 'Próximo'")


def get_element_content(xpath, wait):
    try:
        return wait.until(EC.presence_of_element_located((By.XPATH, xpath))).text
    except StaleElementReferenceException:
        return wait.until(EC.presence_of_element_located((By.XPATH, xpath))).text


def get_page_content(driver, wait):
    # Header
    h1_xpath = "//h1[contains(@class, 'text-4xl')]"
    subtitle_xpath = "//p[@class='mx-auto page-width-wide:mx-0 w-full decoration-primary/6 print:break-inside-avoid max-w-3xl page-api-block:ml-0 text-start justify-self-start']"
    content_xpath = "//div[@class='grid [&>*+*]:mt-5 whitespace-pre-wrap']"
    
    h1 = get_element_content(h1_xpath, wait)
    subtitle = get_element_content(subtitle_xpath, wait)
    content = get_element_content(content_xpath, wait)
    
    result = {
        'title': h1,
        'subtitle': subtitle,
        'content': content,
        'url': driver.current_url,
        'tab_title': driver.title
    }
    
    return result


def upsert_to_df(df: pd.DataFrame, row):
    # Verifica se o title já existe
    if  not row['title'] in df['title'].values:
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
        print(row)

    return df


def teardown(driver):
    '''
    Teardown function to close the WebDriver.
    '''
    driver.quit()

In [122]:
driver, wait = setup(link)
ROW = 0
df = pd.DataFrame(columns=['title', 'subtitle', 'content', 'url', 'tab_title'])

while not verify_end_of_content_page(wait):
    page_content = get_page_content(driver, wait)  # Example data
    upsert_to_df(df, page_content)
    df.loc[ROW] = page_content
    ROW += 1
    navigate_to_next_page(wait)





{'title': 'Bem-vindo à Central de ajuda!', 'subtitle': 'Este é o manual do usuário do FortePlus Sistemas - pulicado no endereço app.forteplus.com.br', 'content': 'Este é o manual do usuário do FortePlus Sistemas - pulicado no endereço app.forteplus.com.br\nComece agora a mudar sua empresa para melhor!\nNavegando no sistema\nConheça os recursos\nCadastrando Produtos\nConheça o menu produtos\nCadastrando Pessoas\nClientes/Fornecedores e mais!\nRealizando Vendas\nPedido de Vendas\nSaiba como transmitir\nNotas Fiscais\nGestão completa\nContas a receber e a pagar ', 'url': 'https://forteplus-sistemas.gitbook.io/forteplus-sistemas', 'tab_title': 'Bem-vindo à Central de ajuda! | FortePlus Sistemas'}
{'title': 'Navegando no sistema', 'subtitle': 'Cada grupo de menu representa um setor responsável pelo controle de processos específicos, como gestão de compras, estoque e vendas, conforme ilustrado na imagem acima.', 'content': "Figura: área de abertura do sistema com links de acessos de acordo c

KeyboardInterrupt: 